# Improve embedding with HuggingFace pre-trained models

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

## 1. Imports

In [1]:
#classic
import pandas as pd
import numpy as np
#models
from transformers import AutoTokenizer, TFAutoModel


/tmp/ipykernel_30146/1325072119.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/isabel/.pyenv/versions/3.10.6/envs/movie_recommendation_GPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Data

In [3]:
df = pd.read_csv('/content/drive/MyDrive/shared_data/raw_data_shared/movie_with_summary.csv')
#df = pd.read_csv('~/code/rossnorman11/raw_data/movie_with_summary.csv')
df.head()

,Unnamed: 0,title,plot_synopsis,gen_summary
0,0,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","""Mr. Holland's Opus"" follows the life of a ded..."
1,1,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","""Scarface"" follows Cuban immigrant Tony Montan..."
2,5,Flightplan,Kyle Pratt (Jodie Foster) is a propulsion engi...,"After her husband's sudden death, a grieving w..."
3,6,Little Caesar,Small-time Italian-American criminals Caesar E...,"""Little Caesar"" follows the rise and fall of a..."
4,7,Savages,The movie begins with a video being shot of me...,Two marijuana entrepreneurs in California are ...


## 4. Using pre-trained models

### 1. Text embedding with bert

In [11]:
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, from_pt = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [12]:
summaries = df['geny_summary'].tolist()

# Tokenize the text data
token_tensor = tokenizer(summaries, padding='max_length', max_length= 500, truncation=True, return_tensors="tf")

# Create input tensors
input_tensor = token_tensor['input_ids']

# Generate embeddings
prediction = model.predict(input_tensor)

194/194 [==============================] - 262s 1s/step


2024-02-08 14:28:07.459148: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1588224000 exceeds 10% of free system memory.


In [14]:
# Process the embeddings as np
summary_embeddings = prediction.last_hidden_state[:, 0, :]

In [ ]:
np.save('/content/drive/MyDrive/shared_data/raw_data_shared/embedding_summary.npy', summary_embeddings)

**Movie similarity with cosinus similarity**

In [15]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Calculate cosine similarity between embeddings
# similarity_matrix = cosine_similarity(embeddings)

# # Example: Get top 10 most similar movies for each movie
# num_movies = similarity_matrix.shape[0]
# top_n = 1  # Number of similar movies to retrieve

# for i in range(num_movies):
#     # Sort similarity scores for movie i
#     sim_scores = sorted(enumerate(similarity_matrix[i]), key=lambda x: x[1], reverse=True)
#     # Exclude movie i itself
#     sim_scores = sim_scores[1:]
#     # Get indices of top similar movies
#     top_indices = [idx for idx, _ in sim_scores[:top_n]]
#     # Print movie titles of top similar movies
#     print(f"Top {top_n} similar movies for '{movie_titles[i]}':")
#     for idx in top_indices:
#         print(f"  - {movie_titles[idx]}")
#     print()

**Movie recommendation with cosine similarity**

In [16]:
# # Calculate Movie Similarity

# similarity_matrix = cosine_similarity(embeddings)

# # Select User Preferences
# user_input = input("Enter a movie title you like: ")

# # Retrieve Similar Movies
# movie_idx = movie_titles.index(user_input)
# sim_scores = list(enumerate(similarity_matrix[movie_idx]))
# sim_scores_sorted = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# # Filter Recommendations (optional)
# top_n = 5  # Number of recommendations to display
# recommended_movies = [movie_titles[idx] for idx, _ in sim_scores_sorted[1:top_n+1]]  # Exclude the input movie itself

# # Display Recommendations
# print(f"Recommended movies based on '{user_input}':")
# for movie in recommended_movies:
#     print(movie)